In [1]:
import os
os.environ["PYTHONIOENCODING"] = "utf-8"

In [2]:
from transformers import pipeline
from openai import OpenAI
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

/Users/majingyi/Downloads/H2M1/.venvH2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

True

In [4]:
def img2text(url):
    img_to_text_pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")
    text = img_to_text_pipe(url)[0]["generated_text"]
    # print(text)
    return text

In [6]:
scenario = img2text("Flowers.jpg")
print(scenario)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/Users/majingyi/Downloads/H2M1/.venvH2/lib/python3.12/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


there is a pink rose that is blooming in the garden


In [7]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    max_tokens=200,
    timeout=None,
    max_retries=2
)

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert gardener. Provide detailed gardening advice. Use the following pieces of retrieved context to generate instructions based on the plants in the given scenario. Use a simple narrative structure to generate these instructions. Keep the response to less than 150 words.",
        ),
        ("human", "{scenario_lang}"),
    ]
)

In [9]:
chain = prompt | llm | StrOutputParser()

chain.invoke({
    "scenario_lang" : scenario
})

"To ensure your pink rose continues to thrive and bloom beautifully, follow these steps:\n\n1. **Watering**: Roses require consistent moisture. Water deeply once a week, ensuring the soil is moist but not waterlogged. Early morning is the best time to water, reducing the risk of fungal diseases.\n\n2. **Pruning**: After blooming, prune spent flowers to encourage new growth. Cut just above a set of five leaves to promote healthy shoots.\n\n3. **Feeding**: Fertilize your rose with a balanced rose fertilizer every 4-6 weeks during the growing season. This will provide essential nutrients for continuous blooming.\n\n4. **Sunlight**: Ensure your rose receives at least 6 hours of sunlight daily. If it's shaded, consider relocating it to a sunnier spot.\n\n5. **Pest and Disease Control**: Regularly inspect for aphids or black spots. Use insecticidal soap or neem oil for pests and remove any diseased leaves promptly.\n\nBy"

In [10]:
def textGeneration_langChain(msg,type):
    """
    msg is the scenario for the text from the pic (hugging face model output);
    type is the genre of the text- Light, Watering, Humidity, Temperture, Pruning, Propagation
    """
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.2,
        max_tokens=200,
        timeout=None,
        max_retries=2
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an expert gardener in giving {suggestion_type} plants caring suggestions. Provide detailed gardening advice regarding {suggestion_type}. Use the following pieces of retrieved context to generate {suggestion_type} suggestions based on the plants in the given scenario. Use a simple narrative structure to generate these suggestions. Keep the suggestions to less than 150 words.",
            ),
            (
                "human", 
                "{scenario_lang}"
            ),
        ]
    )

    chain = prompt | llm | StrOutputParser()

    out_message = chain.invoke({
        "suggestion_type" : type,
        "scenario_lang" : msg,
    })

    return out_message

In [11]:
suggestion = textGeneration_langChain(scenario,"Light")
print(suggestion)

To ensure your pink rose continues to thrive and bloom beautifully, it's important to provide it with the right amount of light. Roses generally require full sun, which means they need at least six to eight hours of direct sunlight each day. Position your rose bush in a spot where it can soak up the morning sun, as this helps to dry dew on the leaves and reduce the risk of fungal diseases.

If your garden has partial shade, try to ensure that the rose receives the most sunlight during the morning and early afternoon. Avoid placing the rose in areas with dense shade, as insufficient light can lead to fewer blooms and weaker growth. Regularly check for any nearby plants or structures that might cast shadows and trim back any overhanging branches to maximize sunlight exposure. With the right light conditions, your pink rose will continue to flourish and brighten your garden.


In [12]:
def runModels_langchain(url, type):
    scenario = img2text(url)
    suggestion = textGeneration_langChain(scenario,type)
    return([scenario,suggestion])

In [13]:
(caption, suggestion) = runModels_langchain("Flowers.jpg","Light")
print(suggestion)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/Users/majingyi/Downloads/H2M1/.venvH2/lib/python3.12/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


To ensure your pink rose continues to thrive and bloom beautifully, it's important to provide it with the right amount of light. Roses generally require full sun, which means they need at least 6 to 8 hours of direct sunlight each day. This exposure helps them produce vibrant blooms and maintain healthy growth.

If your garden has areas with partial shade, try to position the rose where it will receive the morning sun, as this is less intense and can help prevent issues like sunburn on the petals. However, avoid placing the rose in full shade, as this can lead to poor blooming and weak growth.

Regularly check the surrounding plants to ensure they are not overshadowing your rose, and consider pruning nearby foliage if necessary to allow more light to reach it. With the right light conditions, your pink rose will continue to flourish and bring beauty to your garden.
